# Scraping Economic Times

Keywords: ecommerce-fmcg-covid-19

In [1]:
from selenium import webdriver
import time
import nltk
from bs4 import BeautifulSoup
import missingno as msno
from newspaper import Article
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
chrome_path = r'C:\Users\hp\Selenium\ChromeDriver\chromedriver_win32\chromedriver.exe'

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [C:\Users\hp\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


In [9]:
driver.get('https://economictimes.indiatimes.com/topic/ecommerce-fmcg-covid-19/news')

# flag = driver.find_element_by_xpath("//*[@id='listheadline_11585222642782']/a")
# driver.execute_script('arguments[0].scrollIntoView();', flag)

In [11]:
html = driver.page_source.encode('utf-8')
page_num = 0

#WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='categorywise']/div[2]"))).click()
#driver.find_element_by_xpath("//*[@id='categorywise']/div[2]").click()

while driver.find_elements_by_xpath("//*[@id='categorywise']/div[2]"):
    driver.find_element_by_xpath("//*[@id='categorywise']/div[2]").click()
    page_num += 1
    print("getting page number " + str(page_num))
    time.sleep(1)

# //*[@id="categorywise"]/div[2]
# #categorywise > div.autoload_continue                    

getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
getting page number 6


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=85.0.4183.83)


In [12]:
# Selenium script to click on load more for loading every new page, wait 3 seconds for the next batch of
# data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while match==False:
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

# Now that the page is fully scrolled, grab the source code.
source_data = driver.page_source

In [13]:
type(source_data)

str

In [14]:
# Throw your source into BeautifulSoup and start parsing!

len(source_data)

1211987

In [15]:
soup = BeautifulSoup(source_data)
print(soup.prettify())

<html class="win chrome chrome8 webkit webkit5" xmlns:java="java" xmlns:listval="com.indiatimes.cms.utilities.CMSDateUtility" xmlns:valurl="com.times.utilities.CMSWebUtility" xmlns:xhtml="http://www.w3.org/1999/xhtml" xmlns:xslthelper="com.times.utils.xslthelper.XMLHelper">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script async="" src="https://rtbpass-us.andbeyond.media/prebid-et-3.18.0.js" type="text/javascript">
  </script>
  <script async="" src="https://rtbcdn.andbeyond.media/request_js_34387.js" type="text/javascript">
  </script>
  <script async="" crossorigin="anonymous" src="">
  </script>
  <script async="" src="https://survey.survicate.com/workspaces/0be6ae9845d14a7c8ff08a7a00bd9b21/web_surveys.js">
  </script>
  <script src="https://www.googletagservices.com/activeview/js/current/osd.js?cb=%2Fr20100101">
  </script>
  <script async="" src="https://static.clmbtech.com/ad/commons/js/colombia_v2.js" type="text/javascript">
  </script>
  <

In [16]:
article_links = []

for division in soup.find_all('div', attrs={'class': ['clr flt topicstry', 'flr topicstry']}):
    for anch in division.find_all('a', attrs={'class': '', 'target': "_blank"}, href=True):
        if len(anch):
            article_links.append(anch['href'])

In [17]:
len(article_links)

150

In [18]:
type(article_links[0])

str

In [19]:
article_links

['/tech/ites/revival-in-e-commerce-demand-leads-to-increased-hiring-at-bpm-firms/articleshow/77396858.cms',
 '/jobs/e-commerce-comes-to-the-rescue-of-jobless-entry-mid-level-retail-talent/articleshow/77786005.cms',
 '/industry/services/retail/e-commerce-retail-market-expected-to-cross-usd-100-billion-mark-by-2024/articleshow/77786356.cms',
 '/markets/stocks/earnings/dmarts-ecommerce-unit-doubles-sales-but-loss-widenes-too/articleshow/77335502.cms',
 '/tech/internet/e-shoppers-double-amid-pandemic-to-42-of-urban-active-internet-users-study/articleshow/77816032.cms',
 '/news/economy/indicators/e-commerce-spends-surpass-pre-covid-19-levels-post-lockdown-in-delhi-mumbai-bengaluru-report/articleshow/77263286.cms',
 '/industry/services/retail/india-e-commerce-to-grow-27-reliance-to-capture-half-of-online-grocery-sales-goldman/articleshow/77086615.cms',
 '/small-biz/startups/newsbuzz/indias-e-commerce-penetration-will-double-to-11-by-2024-goldman-sachs/articleshow/77189547.cms',
 '/industry/s

In [ ]:
df.columns

In [ ]:
import os
os.chdir('E:/NewsFMCG')
os.getcwd()

# Writing the data to a csv

frame = []
upperframe = []

filename="LiveMint70.csv"
f = open(filename,"w", encoding = 'utf-8')
headers="Date, URL, Newspaper, Title, Article\n"
f.write(headers)

for link in article_links:
    url = 'https://www.livemint.com/' + link
    article = Article(url)
    article.download()
    article.parse()

    nw_date = str(article.publish_date)[:10]
    nw_url = str(article.url)
    nw_paper = 'LiveMint'
    nw_title = article.title
    
    text = article.text
    ult_text = ('').join(text.split('\n'))
    nw_article = ult_text

    frame.append((nw_date, nw_url, nw_paper, nw_title, nw_article))
    f.write(nw_date.replace(",","^") + "," + nw_url + "," + nw_paper + ","
            + nw_title.replace(",","^") + "," 
            + nw_article.replace(",","^") + "\n")
upperframe.extend(frame)

f.close()

data=pd.DataFrame(upperframe, columns=['Date', 'URL', 'Newspaper', 'Title', 'Article'])
data.head()


In [ ]:
msno.matrix(data)

In [ ]:
print(os.path.abspath(filename))